In [5]:
# 使用langgraph接入大模型，使用大模型判断是否相似
# 输入商品名称和top3商品名称，输入相似的结果
from langchain_openai import ChatOpenAI

qwen3_8B = ChatOpenAI(
    model="Qwen/Qwen3-8B",
    base_url="https://api.siliconflow.cn/v1",
    api_key="sk-idaudysppselrwglygkbtatkregsbxhaxypaeulbfpavrals",
)


In [6]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict


class RankSelect(BaseModel):
    """Model for match top rank. Provide json constraints."""

    rank_index: int = Field(
        description="Returns the index of the closest candidate product information. If it is top1_candidate, return 1; if it is top1_candidate, return 2; if it is top1_candidate, return 3; otherwise, return 0.",
    )

In [32]:
rank_prompt = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择最符合输入商品的编号值。
你需要按照下面步骤进行处理和返回：
- 需要先理解输入商品的描述信息origin_product，可以从商品品牌、商品描述、商品名称、商品规格等多个角度进行整理
- 对输入的候选商品信息top1_candidate,top2_candidate,top3_candidate依次进行整理，按照上面四个角度进行匹配，如果无候选商品或候选商品信息明显不符合，请直接返回0。
- 选择最接近的候选商品，返回索引id,范围为1，2，3

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

请使用以下键值以有效的 JSON 格式进行响应：
"rank_index": int，Returns the index of the closest candidate product information. If it is top1_candidate, return 1; if it is top1_candidate, return 2; if it is top1_candidate, return 3; otherwise, return 0.,
"""

In [33]:
model = qwen3_8B.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)


In [42]:
origin_product = "达利园 香奶味软面包  200g／袋 面包就要达利园"
# top1_candidate="达利园 香奶味法式软面包 360g/袋--0.7455--0.6364--1.0000"
# top2_candidate="达利园 桂圆莲子八宝粥 360g/罐--0.4500--0.2143--1.0000"
# top3_candidate="洽洽 香瓜子 160g/袋"
top1_candidate = ""
top2_candidate = "达利园 香奶味法式软面包 360g/袋"
top3_candidate = ""

In [43]:
prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
prompt_format

'\n你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择最符合输入商品的编号值。\n你需要按照下面步骤进行处理和返回：\n- 需要先理解输入商品的描述信息origin_product，可以从商品品牌、商品描述、商品名称、商品规格等多个角度进行整理\n- 对输入的候选商品信息top1_candidate,top2_candidate,top3_candidate依次进行整理，按照上面四个角度进行匹配，如果无候选商品或候选商品信息明显不符合，请直接返回0。\n- 选择最接近的候选商品，返回索引id,范围为1，2，3\n\n以下是输入信息：\n<products_info>\norigin_product：达利园 香奶味软面包  200g／袋 面包就要达利园\ntop1_candidate：\ntop2_candidate：达利园 香奶味法式软面包 360g/袋\ntop3_candidate：\n</products_info>\n\n请使用以下键值以有效的 JSON 格式进行响应：\n"rank_index": int，Returns the index of the closest candidate product information. If it is top1_candidate, return 1; if it is top1_candidate, return 2; if it is top1_candidate, return 3; otherwise, return 0.,\n'

In [44]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)

rankSelect = model.invoke([HumanMessage(content=prompt_format)])

In [45]:
rankSelect.rank_index

2

In [46]:
qwen3_vl = ChatOpenAI(
    model="qwen3-vl-32b-instruct",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key="sk-5c3fb1424b534d19af4d5af9e86dfbfe",
)

In [47]:
match_prompt = """
你是一个电商运营专家，请你根据输入图片url识别理解判断候选商品图片是否与输入商品图片为同一款商品。
你需要按照下面步骤进行处理和返回：
- 需要先识别输入商品图片 origin_product_url，可以从商品品牌、商品数量、商品图片内容等多个角度进行整理
- 对输入的候选商品图片 top1_candidate_url,top2_candidate_url,top3_candidate_url依次匹配，如果匹配不到同款商品直接返回0。
- 选择匹配同款的候选商品，返回索引id,范围为1，2，3

请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int，Returns the index of the match candidate_url. If it is 对输入的候选商品图片top1_candidate_url, return 1; if it is top2_candidate_url, return 2; if it is top3_candidate_url依次匹配, return 3; otherwise, return 0.,
"""

In [56]:

origin_product_url = "http://p0.meituan.net/shangouproductapi/66fe6752d1380cfbcc8767cc16f7b713210389.jpg.webp"
top1_candidate_url = "http://p0.meituan.net/sgopen/d992c7142beea02817c59d42b02d1005367156.jpg.webp"
top2_candidate_url = "http://p0.meituan.net/sgopen/636401ef541c11628b1a5fcd5a1f455a98378.jpg.webp"
top3_candidate_url = "http://p0.meituan.net/sgopen/8d8865c6baba79e0bddbfca947e16273306200.png.webp"

content = [
    {
        "type": "image_url",
        "image_url": {
            "url": origin_product_url,
        },
    },
    {
        "type": "image_url",
        "image_url": {
            "url": top1_candidate_url,
        },
    },
    {
        "type": "image_url",
        "image_url": {
            "url": top2_candidate_url,
        },
    },
    {
        "type": "image_url",
        "image_url": {
            "url": top3_candidate_url,
        },
    },
    {
        "type": "text",
        "text": match_prompt,
    },
]

In [57]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict


class ImageMatchIndex(BaseModel):
    """Model for match top rank. Provide json constraints."""

    match_index: int = Field(
        description="Returns the index of the match candidate_url. If it is 对输入的候选商品图片top1_candidate_url, return 1; if it is top2_candidate_url, return 2; if it is top3_candidate_url依次匹配, return 3; otherwise, return 0.",
    )

In [58]:
image_match_model = qwen3_vl.with_structured_output(ImageMatchIndex).with_retry(stop_after_attempt=3)
image_match_model.invoke([HumanMessage(content=content)])

ImageMatchIndex(match_index=2)